DOwload model


In [27]:
import spacy
import csv
from spacy.matcher import Matcher

In [28]:
try:
    with open("source/Input_File_01.txt", 'rb') as f:
        binary_content = f.read()
except FileNotFoundError:
    print(f"Error: Input file not found at 'source_data/Input_File_01.txt'")

 Read the file in binary mode and split it by the null-byte delimiter (\x00) to get a list of the data fields.

In [29]:
fields = [field.decode('latin-1') for field in binary_content.split(b'\\x00')]

In [30]:

nlp = spacy.load("en_core_web_sm")  # Load English model

# Join the fields into a single text for processing
# (or process chunks of fields that constitute a record)
full_text = "\n".join(fields)

# Process the text with spaCy
doc = nlp(full_text)

print(doc)


1
X530108146000
INTERNAL RETAINING RING
ANNEAU D\'ARRET POUR ALESAGE
INNENHALTERING
ANELLO DI SICUREZZA INTERNO
ANILLO DE RETENCION INTERNO
PIER\xc5\x9aCIE\xc5\x83 USTALAJ\xc4\x84CY WEWN\xc4\x98TRZNY
X530.108.146.000
72311106
DIN472 - 90X3
2
139100150130
SHIM RING
ANNEAU ENTRETOISE
UNTERLEGRING
DISCO SPESSORE
ANILLO DE CALCE
PIER\xc5\x9aCIE\xc5\x83 REGULACYJNY
139.100.150.130
72316227
0,10 MM
139100150131
139.100.150.131
72316228
0,15 MM
139100150132
139.100.150.132
72316229
0,25 MM
139100150133
139.100.150.133
72316230
0,30 MM
139100150134
139.100.150.134
72316231
0,80 MM
3
X548967266000
O-RING
JOINT TORIQUE
GIUNTO DI TENUTA
JUNTA TOROIDAL
X548.967.266.000
72313499
82,14X3,53-NBR 70 FWN56402
4
178100010020
SEALING COVER
CACHE ETANCHE
DICHTDECKEL
COPERTURA TENUTA
CUBIERTA SELLANTE
POKRYWA USZCZELNIAJ\xc4\x84CA
178.100.010.020
72393065
5
X550081705000
SHAFT SEAL
JOINT D\'ARBRE
WELLENDICHTUNG
TENUTA ALBERO
SELLO DE EJE
USZCZELNIENIE WA\xc5\x81U
X550.081.705.000
72430070
FWN65500-DGS40X5

In [31]:


# Tokens
for token in doc:
    print(token.text, token.pos_, token.dep_)



 SPACE dep
1 NUM meta

 SPACE dep
X530108146000 PROPN compound

 SPACE dep
INTERNAL PROPN compound
RETAINING PROPN nmod
RING PROPN compound

 SPACE dep
ANNEAU PROPN dobj
D\'ARRET ADJ amod
POUR PROPN nsubj
ALESAGE VERB ROOT

 SPACE dep
INNENHALTERING NOUN xcomp

 SPACE dep
ANELLO PROPN compound
DI PROPN compound
SICUREZZA PROPN nmod
INTERNO PROPN compound

 SPACE dep
ANILLO PROPN compound
DE PROPN compound
RETENCION PROPN compound
INTERNO PROPN compound

 SPACE dep
PIER\xc5\x9aCIE\xc5\x83 PROPN compound
USTALAJ\xc4\x84CY PROPN compound
WEWN\xc4\x98TRZNY PROPN compound

 SPACE dep
X530.108.146.000 PROPN amod

 SPACE dep
72311106 NUM nummod

 SPACE dep
DIN472 PROPN compound
- PUNCT punct
90X3 PROPN nmod

 SPACE dep
2 NUM nummod

 SPACE dep
139100150130 NUM nummod

 SPACE dep
SHIM PROPN compound
RING PROPN compound

 SPACE dep
ANNEAU PROPN appos
ENTRETOISE NOUN amod

 SPACE dep
UNTERLEGRING NOUN amod

 SPACE dep
DISCO NOUN dobj
SPESSORE VERB dep

 SPACE dep
ANILLO PROPN compound
DE PROPN 

In [32]:

# Named Entities
for ent in doc.ents:
    print(ent.text, ent.label_)

1
X530108146000 QUANTITY
2 CARDINAL
139.100.150.130 CARDINAL
72316227 CARDINAL
MM GPE
139100150131 DATE
139.100.150.131 CARDINAL
72316228 DATE
139100150132
139.100.150.132 CARDINAL
MM GPE
139100150133 DATE
139100150134 DATE
MM GPE
3
X548967266000 QUANTITY
X548.967.266.000 CARDINAL
4 CARDINAL
178100010020 DATE
178.100.010.020 CARDINAL
72393065 DATE
5 CARDINAL
TENUTA ALBERO
 PERSON
SELLO DE EJE ORG
USZCZELNIENIE WA\xc5\x81U
X550.081.705.000
72430070 ORG
FWN65500 ORG
6 CARDINAL
720 CARDINAL
BLOKADA MECHANIZMU ORG
TORNILLO DE CABEZA HEXAGONAL
\xc5\x9aRUBA ORG
X487.535.406.000
72374463 DATE
933 CARDINAL
9 CARDINAL
278110011030 CARDINAL
BRIDA DE COJINETE
KRYZA \xc5\x81O\xc5\xbbYSKA PERSON
72403994 DATE
10
514100010130 CARDINAL
514.100.010.130 CARDINAL
11 CARDINAL
278110011050 DATE
ENGRANAJE LATERAL PERSON
278.110.011.050 CARDINAL
72403996 DATE
22\n<-- CARDINAL
1X CARDINAL
SERBLK ORG
12 CARDINAL
13 CARDINAL
178100010040 DATE
\xc5\x81O\xc5\xbbYSKO POPRZECZNE PERSON
178.100.010.040 CARDINAL
14 

In [33]:
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Noun phrases: ['RING\nANNEAU', '2\n139100150130\nSHIM RING\nANNEAU', 'ANILLO DE CALCE\nPIER\\xc5\\x9aCIE\\xc5\\x83', 'MM', '72316230\n0,30 MM', '139100150134\n139.100.150.134\n72316231\n0,80 MM', 'X548.967.266.000\n72313499\n82,14X3,53-NBR 70 FWN56402\n4\n178100010020\nSEALING COVER\nCACHE', 'POKRYWA', 'USZCZELNIAJ\\xc4\\x84CA', "72393065\n5\nX550081705000\nSHAFT SEAL\nJOINT D\\'ARBRE\nWELLENDICHTUNG\nTENUTA ALBERO\nSELLO DE EJE\nUSZCZELNIENIE WA\\xc5\\x81U\nX550.081.705.000", '72430070\nFWN65500', 'A ROULEMENT CONIQUE\nKEGELROLLENLAGER', 'CUSCINETTO', 'RODAMIENTO DE RODILLOS CONICO\n\\xc5\\x81O\\xc5\\xbbYSKO ROLKOWE STO\\xc5\\xbbKOWE\nX619.049.041.009', 'DIFFERENZIALE\nBLOQUEO DEL DIFERENCIAL\nBLOKADA MECHANIZMU R\\xc3\\x93\\xc5\\xbbNICOWEGO\nH238.104.010.040\n72359217\n8\nX487535406000\nHEXAGONAL', '72374463\nDIN', 'GWA-10.9', 'FLANGE\nFLASQUE ROULEMENT\nLAGERFLANSCH\nFLANGIA DEL CUSCINETTO', 'DE COJINETE\nKRYZA \\xc5\\x81O\\xc5\\xbbYSKA', '278.110.011.030\n72403994\n10\n514100010130